# Evaluation

In [1]:
import sys
import os

repo_path = os.path.sep.join(os.getcwd().split(os.path.sep)[:-3])
print(repo_path)
sys.path.append(repo_path)
%cd ../../..

/home/loebbert/projects/deepclustering/LMU_Master_Practical_SoSe24
/home/loebbert/projects/deepclustering/LMU_Master_Practical_SoSe24


In [2]:
from clustpy.deep.autoencoders import FeedforwardAutoencoder
from practical.DeepClustering.DeepECT.evaluation_pipeline import DatasetType, evaluate

[WARNING] Could not import cv2 in clustpy.data.real_video_data. Please install cv2 by 'pip install opencv-python' if necessary


## Flat Clustering

### MNIST

In [3]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.MNIST, seed=42
)

Autoencoder parameters loaded from file.
cuda


Fit:   1%|          | 524/50000 [00:03<13:21, 61.76it/s] 

Leaf assignments: 48501
Tree size after growing: 5, leaf nodes: 3


Fit:   2%|▏         | 1026/50000 [00:06<11:44, 69.51it/s] 

Leaf assignments: 30418
Tree size after growing: 7, leaf nodes: 4


Fit:   3%|▎         | 1525/50000 [00:09<11:31, 70.10it/s] 

Leaf assignments: 20637
Tree size after growing: 9, leaf nodes: 5


Fit:   3%|▎         | 1666/50000 [00:10<04:47, 168.24it/s]

### USPS

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.USPS, seed=42
)

### Fashion-MNIST

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.FASHION_MNIST, seed=42
)

### Reuters

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.REUTERS, seed=42
)

## Hierarchical Clustering

### MNIST

### USPS

### Fashion-MNIST

### Reuters